In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Conv1D, MaxPooling2D, Activation, Flatten
from keras.utils import np_utils
from sklearn.metrics import precision_recall_fscore_support
from sklearn import preprocessing

/Users/erin/miniconda3/envs/project764/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import math

train_gs_file = './data764/NEEL2016-training_neel.gs'
train_tsv_file = './data764/NEEL2016-training.tsv'
test_gs_file = './data764/NEEL2016-test_neel.gs'
test_tsv_file = './data764/NEEL2016-test.tsv'

#col_names=['tweet_id','start','end','uri', 'confidence', 'type']

with open(train_gs_file) as f:
    train_gs_data = f.readlines()
with open(train_tsv_file) as f:
    train_tsv_data = f.readlines()
with open(test_gs_file) as f:
    test_gs_data = f.readlines()
with open(test_tsv_file) as f:
    test_tsv_data = f.readlines()
train_gs_data[0].split()
train_tweet_dict = {}
for line in train_tsv_data:
    data = line.split('|,|')
    if (len(data) == 2):
        tweet_id = data[0][1:]
        tweet_text = data[1].strip()[:-1]
        if (tweet_id is not None and tweet_text is not None):
            train_tweet_dict[tweet_id] = tweet_text
train_word = []
train_entity = []
for line in train_gs_data:
    data = line.split()
    if (len(data) == 6):
        tweet_id = data[0].strip()
        start_index = int(data[1].strip())
        end_index = int(data[2].strip())
        entity = data[5].strip()
        
        if (entity == 'Organization373937812812615000'):
            entity = 'ORG'
        elif (entity == 'Location'):
            entity = 'LOC'
        elif (entity == 'Person'):
            entity = 'PER'
        elif (entity == 'Organization'):
            entity = 'ORG'
        elif (1):
            entity = 'O'   
        train_entity.append(entity)
        
        word = train_tweet_dict[tweet_id][start_index:end_index]
        train_word.append(word)

In [3]:
test_tweet_dict = {}
for line in test_tsv_data:
    data = line.split('|,|')
    if (len(data) == 2):
        tweet_id = data[0][1:]
        tweet_text = data[1].strip()[:-1]
        if (tweet_id is not None and tweet_text is not None):
            test_tweet_dict[tweet_id] = tweet_text
            
test_word = []
test_entity = []
for line in test_gs_data:
    data = line.split()
    if (len(data) == 6):
        tweet_id = data[0].strip()
        start_index = int(data[1].strip())
        end_index = int(data[2].strip())
        entity = data[5].strip()
        
        if (entity == 'Organization373937812812615000'):
            entity = 'ORG'
        elif (entity == 'Location'):
            entity = 'LOC'
        elif (entity == 'Person'):
            entity = 'PER'
        elif (entity == 'Organization'):
            entity = 'ORG'
        elif (1):
            entity = 'O'  
        test_entity.append(entity)
        
        word = test_tweet_dict[tweet_id][start_index:end_index]
        test_word.append(word)

In [4]:
number_of_labels = set(train_entity+test_entity)

In [5]:
unique_chars = set()
for word in (train_word + test_word):
    for char in word:
        unique_chars.add(char)

In [6]:
char_dict = {}
for i, char in enumerate(unique_chars):
    char_dict[char] = i + 1

In [7]:
def convert_word_2_vec(input_words):
    output_words = []
    for word in input_words:
        word_vec = []
        for char in word:
            word_vec.append(char_dict[char])
        output_words.append(word_vec)
    return output_words

In [8]:
train_word_vectors = convert_word_2_vec(train_word)
test_word_vectors = convert_word_2_vec(test_word)

In [9]:
# find word of maximum length
max_word_length = 0
for word in (train_word + test_word):
    if len(word) > max_word_length:
        max_word_length = len(word)

In [10]:
train_one_hot = preprocessing.LabelEncoder().fit_transform(train_entity)
train_one_hot = np_utils.to_categorical(train_one_hot)
test_one_hot = preprocessing.LabelEncoder().fit_transform(test_entity)
test_one_hot = np_utils.to_categorical(test_one_hot)

In [11]:
train_words_padded = sequence.pad_sequences(train_word_vectors, maxlen=max_word_length)
test_words_padded = sequence.pad_sequences(test_word_vectors, maxlen=max_word_length)
train_entities = np.array(train_one_hot)
test_entities = np.array(test_one_hot)
#train_feature = np.expand_dims(np.stack(train_words_padded, axis=1), axis=1)
#test_feature = np.expand_dims(np.stack(test_words_padded, axis=1), axis=1)
train_feature = np.expand_dims(train_words_padded, axis=2)
test_feature = np.expand_dims(test_words_padded, axis=2)
#train_feature = train_words_padded
#test_feature = test_words_padded

In [20]:
#CNN
batch_size = 128
input_shape=(52,1)
epochs = 10
model = Sequential()
model.add(Conv1D(32, kernel_size=3, strides=1, padding='same', activation='relu',input_shape=input_shape))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 1)))
model.add(Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)), strides=(2, 1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(units=4, activation='softmax'))

'''model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.05),
              metrics=['accuracy'])
'''

model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model.fit(train_feature, train_entities,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(test_feature, test_entities)
          )

Train on 8665 samples, validate on 1022 samples
Epoch 1/10
8665/8665 [==============================] - 2s 231us/step - loss: 1.4684 - acc: 0.3927 - val_loss: 1.3738 - val_acc: 0.2720
Epoch 2/10
8665/8665 [==============================] - 2s 184us/step - loss: 1.1939 - acc: 0.4836 - val_loss: 1.3717 - val_acc: 0.3669
Epoch 3/10
8665/8665 [==============================] - 2s 183us/step - loss: 1.1358 - acc: 0.5076 - val_loss: 1.2400 - val_acc: 0.4178
Epoch 4/10
8665/8665 [==============================] - 2s 186us/step - loss: 1.0760 - acc: 0.5393 - val_loss: 1.3443 - val_acc: 0.4902
Epoch 5/10
8665/8665 [==============================] - 2s 184us/step - loss: 1.0171 - acc: 0.5761 - val_loss: 1.2284 - val_acc: 0.4863
Epoch 6/10
8665/8665 [==============================] - 2s 184us/step - loss: 0.9657 - acc: 0.6048 - val_loss: 1.2598 - val_acc: 0.4002
Epoch 7/10
8665/8665 [==============================] - 2s 195us/step - loss: 0.9176 - acc: 0.6265 - val_loss: 1.3167 - val_acc: 0.4061


In [16]:
set(train_entity)



{'LOC', 'O', 'ORG', 'PER'}

In [18]:
test_prediction = model.predict(test_feature)

test_pred= []
for pred in test_prediction:
    index = np.argmax(pred)
    if index == 0:
        test_pred.append('LOC')
    elif index == 1:
        test_pred.append('O')
    elif index == 2:
        test_pred.append('ORG')
    elif index == 3:
        test_pred.append('PER')
   

In [19]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(test_entity, test_pred)

(array([0.08955224, 0.47265625, 0.32876712, 0.38663484]),
 array([0.41860465, 0.25      , 0.30379747, 0.48071217]),
 array([0.14754098, 0.32702703, 0.31578947, 0.42857143]),
 array([ 43, 484, 158, 337]))